### Generate a guess

In [1]:
import random

def generate_parent(length):
    genes=[]
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

`random.sample` takes `sampleSize` values from the input without replacement. This means that there will be no duplicates in the generated parent unless `geneSet` contains duplicates, or length is greater than `len(geneSet)`. The implementation above can generate a long string with a small set of genes and uses as many unique genes as possible.

### Fitness

- The fitness value is the only feedback that guides GA to a solution.
- In this project, the fitness value is the total number of letters in the guess that match the letter in the same position of the password.

In [2]:
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target,guess) if expected == actual)

### Mutate